In [1]:
"""
In this experiment, we decode a Shor's nine-qubit quantum error correcting code.
We show decoding of the Shor's nine-qubit code using Dephasing DMRG --
our own built-in DMRG-like optimisation algorithm to solve the main component problem --
the problem of finding a computational basis state cotributing the most to a given state.
"""

"\nIn this experiment, we decode a Shor's nine-qubit quantum error correcting code.\nWe show decoding of the Shor's nine-qubit code using Dephasing DMRG --\nour own built-in DMRG-like optimisation algorithm to solve the main component problem --\nthe problem of finding a computational basis state cotributing the most to a given state.\n"

In [2]:
import numpy as np
import qecstruct as qec

from examples.decoding.decoding import IDENTITY, XOR_BULK, XOR_LEFT, XOR_RIGHT, SWAP, COPY
from examples.decoding.decoding import bin_vec_to_dense, ConstraintString, get_constraint_sites, apply_parity_constraints, get_codewords, prepare_codewords, apply_bias_channel, decode, linear_code_checks

from mdopt.mps.canonical import CanonicalMPS
from mdopt.mps.utils import create_simple_product_state, create_custom_product_state, mps_from_dense, create_state_vector, find_orth_centre, inner_product
from mdopt.utils.utils import mpo_to_matrix
from functools import reduce
from opt_einsum import contract

In [3]:
code = qec.shor_code()
code

X stabilizers:
[0, 1, 2, 3, 4, 5]
[3, 4, 5, 6, 7, 8]
Z stabilizers:
[0, 1]
[1, 2]
[3, 4]
[4, 5]
[6, 7]
[7, 8]

In [4]:
num_sites = 2 * len(code) + code.num_x_logicals() + code.num_z_logicals()
assert num_sites == 20
string = "0" * num_sites
# not perturbing anything for now, should get an identity logical in the end
error_state = create_custom_product_state(string=string, form="Right-canonical")

In [5]:
def linear_csscode_checks(code: qec.CssCode) -> tuple[list[np.ndarray]]:
    """
    Given a CSS code, returns a list of its checks, where each check
    is represented as a list of indices of the bits adjacent to it.

    Parameters
    ----------
    code : qec.CssCode
        The CSS code object.

    Returns
    -------
    checks : tuple[list[np.ndarray]]
        The checks.
    """

    parity_matrix_x = code.x_stabs_binary()
    array_x = np.zeros((parity_matrix_x.num_rows(), parity_matrix_x.num_columns()), dtype=int)
    for row, cols in enumerate(parity_matrix_x.rows()):
        for col in cols:
            array_x[row, col] = 1
    
    parity_matrix_z = code.z_stabs_binary()
    array_z = np.zeros((parity_matrix_z.num_rows(), parity_matrix_z.num_columns()), dtype=int)
    for row, cols in enumerate(parity_matrix_z.rows()):
        for col in cols:
            array_z[row, col] = 1

    return (
            [
                2*np.nonzero(row)[0] for row in array_x
                ],
            [
                2*np.nonzero(row)[0] + 1 for row in array_z
                ],
    )

In [6]:
checks_x, checks_z = linear_csscode_checks(code)
print(checks_x)
print(checks_z)

[array([ 0,  2,  4,  6,  8, 10]), array([ 6,  8, 10, 12, 14, 16])]
[array([1, 3]), array([3, 5]), array([7, 9]), array([ 9, 11]), array([13, 15]), array([15, 17])]


In [7]:
def get_cssconstraint_sites(code: qec.CssCode) -> tuple[list[int]]:
    """
    Returns the list of MPS sites where the logical constraints should be applied.

    Parameters
    ----------
    code : qec.CssCode
        The CSS code object.

    Returns
    strings : tuple[list[list[int]]]
        List of MPS sites.
    """

    sites_x, sites_z = linear_csscode_checks(code)

    #check_x_degree = len(sites_x[0])
    constraints_strings_x = []

    #check_z_degree = len(sites_z[0])
    constraints_strings_z = []

    for sites in sites_x:

        # Retreiving the sites indices where we apply the "bulk"/"boundary" XOR tensors.
        xor_left_sites_x = [sites[0]]
        xor_bulk_sites_x = [sites[i] for i in range(1, len(sites) - 1)]
        xor_right_sites_x = [sites[-1]]

        # Retreiving the sites indices where we apply the SWAP tensors.
        swap_sites_x = list(range(sites[0] + 1, sites[-1]))
        for k in range(1, len(sites) - 1):
            swap_sites_x.remove(sites[k])

        constraints_strings_x.append(
            [xor_left_sites_x, xor_bulk_sites_x, swap_sites_x, xor_right_sites_x]
        )

    for sites in sites_z:

        # Retreiving the sites indices where we apply the "bulk"/"boundary" XOR tensors.
        xor_left_sites_z = [sites[0]]
        xor_bulk_sites_z = [sites[i] for i in range(1, len(sites) - 1)]
        xor_right_sites_z = [sites[-1]]

        # Retreiving the sites indices where we apply the SWAP tensors.
        swap_sites_z = list(range(sites[0] + 1, sites[-1]))
        for k in range(1, len(sites) - 1):
            swap_sites_z.remove(sites[k])

        constraints_strings_z.append(
            [xor_left_sites_z, xor_bulk_sites_z, swap_sites_z, xor_right_sites_z]
        )

    return constraints_strings_x, constraints_strings_z

In [8]:
tensors = [XOR_LEFT, XOR_BULK, SWAP, XOR_RIGHT]
csscode_constraint_sites = get_cssconstraint_sites(code)

In [9]:
# X checks' tensors
for s in csscode_constraint_sites[0]:
    print(s)

[[0], [2, 4, 6, 8], [1, 3, 5, 7, 9], [10]]
[[6], [8, 10, 12, 14], [7, 9, 11, 13, 15], [16]]


In [10]:
# Z checks' tensors
for s in csscode_constraint_sites[1]:
    print(s)
# No bulk XOR's indeed

[[1], [], [2], [3]]
[[3], [], [4], [5]]
[[7], [], [8], [9]]
[[9], [], [10], [11]]
[[13], [], [14], [15]]
[[15], [], [16], [17]]


In [11]:
print(code.x_logicals_binary()) # Z_log = X_0 X_1 X_2
print(code.z_logicals_binary()) # X_log = Z_0 Z_3 Z_6 (qecstruct naming?)

[0, 1, 2]

[0, 3, 6]



In [12]:
def csscode_logicals(code: qec.CssCode):
    """
    Returns the list of MPS sites where the logical constraints should be applied.

    Parameters
    ----------
    code : qec.CssCode
        The CSS code object.

    Returns
    logicals : tuple[list[int]]
        List of logical operators, first X, then Z.
    """

    log_matrix_x = code.z_logicals_binary()
    array_x = np.zeros((log_matrix_x.num_rows(), log_matrix_x.num_columns()), dtype=int)
    for row, cols in enumerate(log_matrix_x.rows()):
        for col in cols:
            array_x[row, col] = 1

    log_matrix_z = code.x_logicals_binary()
    array_z = np.zeros((log_matrix_z.num_rows(), log_matrix_z.num_columns()), dtype=int)
    for row, cols in enumerate(log_matrix_z.rows()):
        for col in cols:
            array_z[row, col] = 1

    x_logical = [2*np.nonzero(row)[0] + 1 for row in array_x]
    z_logical = [2*np.nonzero(row)[0] for row in array_z]

    return x_logical[0], z_logical[0]

In [13]:
print(csscode_logicals(code)[0]) # X
print(csscode_logicals(code)[1]) # Z

[ 1  7 13]
[0 2 4]


In [14]:
def get_csslogicals_sites(code: qec.CssCode) -> tuple[list[int]]:
    """
    Returns the list of MPS sites where the logical operators should be applied.

    Parameters
    ----------
    code : qec.CssCode
        The CSS code object.

    Returns
    -------
    strings : tuple[list[int]]
        List of MPS sites.
    """

    sites_x, sites_z = csscode_logicals(code)

    # Retreiving the sites indices where we apply the COPY tensors.
    copy_site_x = [2 * (len(code)) + code.num_x_logicals() + code.num_z_logicals() - 2]
    copy_site_z = [2 * (len(code)) + code.num_x_logicals() + code.num_z_logicals() - 1]

    # Retreiving the sites indices where we apply the left XOR tensors.
    xor_left_site_x = [sites_x[0]]
    xor_left_site_z = [sites_z[0]]

    # Retreiving the sites indices where we apply the bulk XOR tensors.
    xor_bulk_sites_x = [sites_x[i] for i in range(1, len(sites_x))]
    xor_bulk_sites_z = [sites_z[i] for i in range(1, len(sites_z))]

    # Retreiving the sites indices where we apply the SWAP tensors.
    swap_sites_x = list(range(sites_x[0]+1, copy_site_x[0]))
    swap_sites_x = [site for site in swap_sites_x if site not in xor_bulk_sites_x]
    swap_sites_z = list(range(sites_z[0]+1, copy_site_z[0]))
    swap_sites_z = [site for site in swap_sites_z if site not in xor_bulk_sites_z]

    string_x = [xor_left_site_x, xor_bulk_sites_x, swap_sites_x, copy_site_x]
    string_z = [xor_left_site_z, xor_bulk_sites_z, swap_sites_z, copy_site_z]

    return string_x, string_z

In [15]:
print(get_csslogicals_sites(code)[0]) # X
print(get_csslogicals_sites(code)[1]) # Z

[[1], [7, 13], [2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 14, 15, 16, 17], [18]]
[[0], [2, 4], [1, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], [19]]


In [16]:
tensors = [XOR_LEFT, XOR_BULK, SWAP, XOR_RIGHT]
csscode_constraint_sites = get_cssconstraint_sites(code)

tensors_for_logicals = [XOR_LEFT, XOR_BULK, SWAP, COPY]
constraint_sites_logicals = get_csslogicals_sites(code)

In [17]:
error_state = apply_parity_constraints(
    error_state, csscode_constraint_sites[0], tensors, renormalise=False
    )

100%|██████████| 2/2 [00:00<00:00, 47.09it/s]


In [18]:
error_state = apply_parity_constraints(
    error_state, csscode_constraint_sites[1], tensors, renormalise=False
    )

100%|██████████| 6/6 [00:00<00:00, 290.64it/s]


In [19]:
error_state = apply_parity_constraints(
    error_state, constraint_sites_logicals, tensors_for_logicals, renormalise=False
    )

100%|██████████| 2/2 [00:00<00:00, 41.24it/s]


In [20]:
marg = error_state.marginal(sites_to_marginalise=list(range(18)))

20 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
visiting site  0  with bond_dim= 1
tensor  1  inserting into tensor 0
new list of sites to marginalise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
tensors' shapes after a round of bubbling
[(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 2, 1), (1, 2, 1)]
visiting site  0  with bond_dim= 1
tensor  1  inserting into tensor 0
new list of sites to marginalise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
tensors' shapes after a round of bubbling
[(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 2, 1), (1, 2, 1)]
visiting site  0  with bond_dim= 1
tensor  1  inserting into tensor 0
new list of sites to marginalise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
tensors' shapes after a round of bubbling
[(1, 1), (1, 1), (1, 

In [21]:
marg.dense()

array([0.00195312, 0.        , 0.        , 0.        ])